In [ ]:
import pyspark
from pyspark.sql import SparkSession

# --- 1. CONFIGURAÇÃO CORRETA PARA SPARK 4.0.x ---
# Versões compatíveis para o seu ambiente
delta_version = "4.0.0"  # A versão do Delta Lake compatível com Spark 4.0
scala_version = "2.13"   # Spark 4.0 usa Scala 2.13

print(f"Configurando Spark para usar Delta Lake v{delta_version} com Scala v{scala_version}...")

spark = (
    SparkSession.builder
    .appName("RestauranteDeltaCorreto")
    .config("spark.jars.packages", f"io.delta:delta-spark_{scala_version}:{delta_version}")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

print(f"Sessão Spark v{spark.version} iniciada com sucesso!")
print("-" * 50)


# --- 2. CRIAÇÃO DA TABELA (SE NÃO EXISTIR) ---
print("1. Criando a tabela 'pratos_delta' se ela não existir...")

spark.sql("""
  CREATE TABLE IF NOT EXISTS default.pratos_delta (
    id INT,
    nome STRING,
    categoria STRING,
    ingredientes ARRAY<STRING>,
    preco DECIMAL(10, 2),
    disponivel BOOLEAN
  ) USING delta
  COMMENT 'Tabela com o cardápio do restaurante'
""")

print("   -> Tabela pronta para uso.")
print("-" * 50)


# --- 3. PREENCHIMENTO DA TABELA (APENAS SE ESTIVER VAZIA) ---
print("2. Verificando se a tabela precisa ser populada...")

if spark.table("default.pratos_delta").isEmpty():
    print("   -> A tabela está vazia. Inserindo dados iniciais...")
    spark.sql("""
      INSERT INTO default.pratos_delta VALUES
      (1, 'Spaghetti Carbonara', 'Massas', ARRAY['Spaghetti', 'Ovo', 'Guanciale', 'Parmesão'], 55.00, true),
      (2, 'Risotto de Cogumelos', 'Risotos', ARRAY['Arroz Arbóreo', 'Cogumelo Porcini', 'Vinho Branco', 'Caldo de Legumes'], 62.50, true)
    """)
    print("   -> Dados inseridos com sucesso!")
else:
    print("   -> A tabela já contém dados. Nenhuma inserção foi feita.")

print("-" * 50)


# --- 4. EXIBIÇÃO DO RESULTADO FINAL ---
print("\n3. Exibindo o conteúdo final da tabela 'pratos_delta':")
spark.sql("SELECT * FROM default.pratos_delta ORDER BY id").show(truncate=False)

Iniciando a sessão Spark com suporte a Delta Lake...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/02 13:12:47 WARN Utils: Your hostname, PMC104684, resolves to a loopback address: 127.0.1.1; using 172.25.139.234 instead (on interface eth0)
25/10/02 13:12:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/viniciu/.cache/pypoetry/virtualenvs/my-pyspark-lab-qqAYommE-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/viniciu/.ivy2.5.2/cache
The jars for the packages stored in: /home/viniciu/.ivy2.5.2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ad0f39ab-5a81-483a-a995-56fdb6d73f8a;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5

Sessão Spark v4.0.1 iniciada com sucesso!
--------------------------------------------------
1. Criando a tabela 'pratos_delta' se ela não existir...


Py4JJavaError: An error occurred while calling o39.sql.
: java.lang.NoClassDefFoundError: scala/collection/GenTraversableOnce
	at java.base/java.lang.Class.getDeclaredConstructors0(Native Method)
	at java.base/java.lang.Class.privateGetDeclaredConstructors(Class.java:3373)
	at java.base/java.lang.Class.getConstructor0(Class.java:3578)
	at java.base/java.lang.Class.getDeclaredConstructor(Class.java:2754)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:64)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:90)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:469)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:90)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:54)
	at org.apache.spark.sql.connector.cat